In [5]:
! pip install dash plotly pandas
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load the gyroscope data from CSV
# Ensure you replace 'gyroscope_data.csv' with your actual CSV file.
df = pd.read_csv('gyro_data.csv')

# Initialize Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Gyroscope Data Visualization"),
    
    # Dropdown for selecting variable to plot (x, y, z or all)
    html.Label("Select Gyroscope Variable(s)"),
    dcc.Dropdown(
        id='variable-dropdown',
        options=[
            {'label': 'X', 'value': 'gyro_x'},
            {'label': 'Y', 'value': 'gyro_y'},
            {'label': 'Z', 'value': 'gyro_z'},
            {'label': 'All', 'value': 'all'}
        ],
        value='all',  # default value
        multi=False  # Change to True if you want to allow multiple selections
    ),
    
    # Input for selecting number of data samples
    html.Label("Number of Samples"),
    dcc.Input(id='sample-input', type='number', value=100),  # default value
    
    # Button to navigate previous/next data samples
    html.Button('Previous', id='prev-button', n_clicks=0),
    html.Button('Next', id='next-button', n_clicks=0),
    
    # Graph to show the gyroscope data
    dcc.Graph(id='gyro-graph'),
    
    # Table to display summary statistics
    html.Div(id='summary-table')
])

# Callback to update graph based on user input
@app.callback(
    Output('gyro-graph', 'figure'),
    Output('summary-table', 'children'),
    Input('variable-dropdown', 'value'),
    Input('sample-input', 'value'),
    Input('prev-button', 'n_clicks'),
    Input('next-button', 'n_clicks')
)
def update_graph(variable, num_samples, prev_clicks, next_clicks):
    # Calculate which part of the data to show
    total_samples = len(df)
    offset = (prev_clicks - next_clicks) * num_samples
    
    # Ensure offset is within valid range
    offset = max(0, min(offset, total_samples - num_samples))
    
    # Get the data slice
    data_slice = df[offset:offset + num_samples]
    
    # Prepare figure based on the selected variable
    if variable == 'all':
        fig = px.line(data_slice, x=data_slice.index, y=['gyro_x', 'gyro_y', 'gyro_z'], 
                      labels={'value': 'Gyroscope Values', 'index': 'Sample Number'})
    else:
        fig = px.line(data_slice, x=data_slice.index, y=variable, 
                      labels={variable: f'{variable} Value', 'index': 'Sample Number'})
    
    # Prepare the statistical summary
    summary = data_slice.describe().to_dict()
    summary_table = html.Table([
        html.Tr([html.Th(col) for col in summary.keys()]),
        html.Tr([html.Td(str(summary[col]['mean'])) for col in summary.keys()]),
    ])
    
    return fig, summary_table


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
